In [1]:
#| default_exp player

# Player

> Easily analyze and compare pickleball players

In [1]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [111]:
#| export
from pklshop.data import *
from pklshop.stats import *
from pklshop.name import *
from pklshop.game import *
from pklshop.match import *
from pklshop.team import *
import pandas as pd
import numpy as np

In [113]:
#! Need to think about how to store these tables so that functions can use them
#! but don't want to have to pass specific tables every time
rally = get_tab_as_df("rally")
players = get_tab_as_df("player")
game = get_tab_as_df("game")
team = get_tab_as_df("team")
match = get_tab_as_df("match")
tournament = get_tab_as_df("tournament") 

In [268]:
#| export
class Player():

    def __init__(self, player_id: str):
        self.player_id = player_id
        self.name = get_player_name(self.player_id)
        self.teams = self.associated_teams()

        self.matches_played = self.matches_played()
        self.num_matches_played = len(self.matches_played)
        self.matches_won = self.matches_won()
        self.num_matches_won = len(self.matches_won)

        self.games_played = self.games_played()
        self.num_games_played = len(self.games_played)
        self.games_won = self.games_won()
        self.num_games_won = len(self.games_won)

        self.partner_ids = self.associated_partners()

    def __str__(self):
        return f"{self.name}, id: {self.player_id}"
    __repr__ = __str__

    def associated_teams(self):
        '''
        Returns the team_ids of the teams that the player played for.
        '''
        return team[team.player_id == self.player_id].team_id.values
    
    def associated_partners(self):
        '''
        Returns the player_ids of the players that the player has played with
        '''
        partners = []
        for team_id in self.teams:
            team = Team(team_id)
            for p_id in team.players:
                if p_id != self.player_id:
                    partners.append(p_id)
        return partners
    
    def games_played(self):
        '''
        Returns the game_ids of games played (in the database) by the player
        '''
        gs = []
        for team_id in self.teams:
            team_games = game[(game.w_team_id == team_id) | (game.l_team_id == team_id)].game_id.values
            gs += team_games.tolist()
        return gs
    
    def matches_played(self):
        '''
        Returns the match_ids of matches played (in the database) by the player
        '''
        ms = []
        for team_id in self.teams:
            team_matches = match[(match.team_id_1 == team_id) | (match.team_id_2 == team_id)].match_id.values
            ms += team_matches.tolist()
        return ms
        
    def matches_won(self):
        ms = []
        for m_id in self.matches_played:
            m = Match(m_id)
            if m.w_team_id in self.teams:
                ms += [m_id]
        return ms
    
    def games_won(self):
        gs = []
        for g_id in self.games_played:
            g = Game(g_id)
            if g.w_team_id in self.teams:
                gs += [g_id]
        return gs
    
    def error_rate(self):
        '''
        Returns the error rate of the player accross all games played
        '''
        error_rates = []
        for g_id in self.games_played:
            g = Game(g_id)
            error_rate = g.get_error_rate(self.player_id)
            error_rates += [error_rate]
        return sum(error_rates)/len(error_rates)
    
    def partners_win_rate(self):
        '''
        Returns the win rate of the player's partners excluding games where the player was on the same team as their partner
        '''
        
        partners_exclusive_wins = 0

        for p_id in self.partner_ids:
            p2 = Player(p_id)
            games_w_partner = self.games_played_with_partner(p2)
            p2_games_wo_player_partner = set(self.games_played).difference(set(games_w_partner))

            for g_id in p2_games_wo_player_partner:
                g = Game(g_id)
                if g.w_team_id in p2.teams:
                    partners_exclusive_wins += 1
                
        if len(p2_games_wo_player_partner) == 0:
            return None

        return partners_exclusive_wins/len(p2_games_wo_player_partner)

    def partners_error_rate(self):
        '''
        Returns the average error rate of the player's partners
        '''
        error_rates = []
        for p_id in self.partner_ids:
            p = Player(p_id)
            error_rates += [p.error_rate()]
        return sum(error_rates)/len(error_rates)
            
    def games_played_with_partner(self, partner):
        '''
        Returns the games played on the same team as the given partner
        '''
        games_in_common = set(self.games_played).intersection(set(partner.games_played))
        games_w_partner = []
        for g_common in games_in_common:
            g = Game(g_common)
            #find if self and partner were on the same team
            if (g.w_team_id in self.teams and g.w_team_id in partner.teams) or (g.l_team_id in self.teams and g.l_team_id in partner.teams):
                games_w_partner.append(g.game_id)
        return games_w_partner

    def third_shot_profile(self):
        '''
        Returns the third shot profile of the player
        '''
        third_shots = np.zeros(4)
        for g_id in self.games_played:
            g = Game(g_id)
            thirds = np.array(g.player_third_shots(self.player_id))
            third_shots += thirds
        return third_shots

    def summarize_player(self):
        '''
        Prints a summary of the player's stats
        '''
        print(f"Player: {self.name}")
        print(f"Matches played: {self.num_matches_played}")
        print(f"Matches won: {self.num_matches_won}")
        print("Percentage of matches won: {:.2f}%".format(self.num_matches_won/self.num_matches_played*100))
        print(f"Games played: {self.num_games_played}")
        print(f"Games won: {self.num_games_won}")
        print("Percentage of games won: {:.2f}%".format(self.num_games_won/self.num_games_played*100))
        print(f"Teams: {[get_team_name(team) for team in self.teams]}")


In [253]:
team.columns

Index(['team_id', 'player_id', 'player_seq_nbr', 'team_nm', 'maint_dtm',
       'maint_app', 'create_dtm', 'create_app'],
      dtype='object')

In [269]:
p = Player("P1")
p

Jesse Irvine, id: P1

In [274]:
p.partners_error_rate()

0.06830721826632076

In [270]:
p2 = Player("P2")
p2

Catherine Parenteau, id: P2

In [271]:
games_in_common = set(p.games_played).intersection(set(p2.games_played))
for g_id in games_in_common:
    g = Game(g_id)
    print(g,g.w_team_id, g.l_team_id)


Game(G3) T2 T1
Game(G2) T1 T2
Game(G1) T2 T1
Game(G23) T21 T20


In [272]:
print(p2.games_played)
print(p.games_played)
set(p2.games_played).difference(p.games_played_with_partner(p2))

['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G22', 'G23']
['G1', 'G2', 'G3', 'G14', 'G15', 'G23']


{'G22', 'G23', 'G4', 'G5', 'G6'}

In [273]:
p3 = Player("P3")
p3.partners_win_rate()

In [260]:
p.partners_win_rate()

0.0

In [144]:
p.third_shot_profile()

array([67., 22.,  0., 89.])

In [58]:
p.summarize_player()

Player: Jesse Irvine
Matches played: 3
Matches won: 2
Percentage of matches won: 66.67%
Games played: 6
Games won: 4
Percentage of games won: 66.67%
Teams: ['Jesse Irvine & Catherine Parenteau', 'Jesse Irvine & Anna Bright', 'Jesse Irvine & Lucy Kovalova']


In [141]:
print(p.games_played)
print(p.matches_played)

['G1', 'G2', 'G3', 'G14', 'G15', 'G23']
['M1', 'M7', 'M14']


In [203]:
#| hide
test_eq(p.num_games_played, len(p.games_played))
test_eq(p.num_matches_played, len(p.matches_played))

In [207]:
#| export
def head_to_head(p1: Player, p2: Player):
    '''
    Returns the results of matches where p1 and p2 have played against each other
    '''
    p1_matches = p1.matches_played
    p2_matches = p2.matches_played

    p1_count = 0
    p2_count = 0
    for m_id in p1_matches:
        if m_id in p2_matches:
            m = Match(m_id)
            if (p.name in m.w_team_name and p2.name in m.l_team_name):
                p1_count += 1
            elif (p.name in m.l_team_name and p2.name in m.w_team_name):
                p2_count += 1
    print(f"{p1.name} has played against {p2.name} in {p1_count + p2_count} matches and has won {p1_count} times")

In [209]:
head_to_head(p,p2)

Jesse Irvine has played against Catherine Parenteau in 1 matches and has won 1 times


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()